In [46]:
import numpy as np
import pickle as pkl
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from hmmlearn import hmm
from statsmodels.tsa.arima.model import ARIMA
from utils import softmax_weighting, get_mae_errors, get_mape_errors, get_mse_errors, get_rmse_errors, normalized_inverse_of_errors_weighting
from sklearn.preprocessing import StandardScaler

In [9]:
df=pd.read_csv("cleaned_ECG.csv")
df.drop(columns=['ECG2'],inplace=True) #considering only one ECG signal for forecasting
df.columns = ['ds', 'y']
df.shape

(7680, 2)

In [11]:
quantiles = [0.015, 0.985]

params = {
    "n_lags": 24,
    "n_forecasts": 7,
    "n_changepoints": 20,
    "learning_rate": 0.01,
    "ar_layers": [32, 16, 16, 32],
    "epochs": 70,
    "batch_size": 64,
    "quantiles": quantiles,
}


m = NeuralProphet(**params)
m.set_plotting_backend("plotly-static")
set_log_level("ERROR")

In [12]:
df_train, df_test = m.split_df(df, valid_p=0.05, local_split=True)
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (7292, 2)
Test shape: (412, 2)


In [ ]:
def grid_search_arima(data, p_range, d_range, q_range):
    best_score = float('inf')
    best_order = None
    
    for p in p_range:
        for d in d_range:
            for q in q_range:
                try:
                    model = ARIMA(data, order=(p,d,q))
                    results = model.fit()
                    aic = results.aic
                    if aic < best_score:
                        best_score = aic
                        best_order = (p,d,q)
                except:
                    continue
    return best_order

In [ ]:
best_order = grid_search_arima(df_train['y'], 
                             p_range=range(0,3), 
                             d_range=range(0,2), 
                             q_range=range(0,3))
print(f"Best ARIMA order: {best_order}")

In [27]:
from tensorflow.keras.models import load_model
lstm_model = load_model("model_store/lstm_ecg.keras")
lstm_model.summary()

with open("model_store/best_order_ecg.pkl", "rb") as f:
    loaded_order = pkl.load(f)

with open("model_store/opt_no_states_ecg.pkl", "rb") as f:
    opt_states = pkl.load(f)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 75)             │        23,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            76 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,530 (271.61 KB)

 Trainable params: 23,176 (90.53 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 46,354 (181.07 KB)

In [48]:
scaler = StandardScaler()
scaler.fit(np.array(df['y']).reshape(-1,1))

def dynamic_ensemble_prediction(train, test):
    train_hmm = train.reshape(-1,1)
    train_hmm = scaler.transform(train_hmm)
    test_hmm = test.reshape(-1,1)
    test_hmm = scaler.transform(test_hmm)
    hmm_history = train_hmm
    history = np.array(train)

    predictions = []
    truth_values = []
    lstm_preds = []
    hmm_preds = []
    arima_preds = []

    for i in range(len(test)):
        print(f'{i+1}/{len(test)}')
        truth_values.append(test[i])
        # LSTM
        lstm_pred = lstm_model.predict(history[-24:].reshape(1,24))[0][0]
        lstm_preds.append(lstm_pred)
        # ARIMA
        arima_model = ARIMA(history, order=loaded_order)
        arima_fit = arima_model.fit()
        arima_pred = arima_fit.forecast(steps=1)[0]
        arima_preds.append(arima_pred)
        # HMM
        hmm_model = hmm.GaussianHMM(n_components=opt_states, covariance_type='diag', tol=0.0001, n_iter=100)
        hmm_model.fit(hmm_history)
        hidden_states = hmm_model.predict(hmm_history)
        last_hidden_state = hidden_states[-1]
        next_state_probs = hmm_model.transmat_[last_hidden_state]
        predicted_state = np.argmax(next_state_probs)
        predicted_value = hmm_model.means_[predicted_state][0]
        hmm_pred = scaler.inverse_transform(np.array(predicted_value).reshape(-1,1))[0][0]
        hmm_preds.append(hmm_pred)

        #Error Measurement
        arima_error = get_mae_errors(arima_preds, truth_values)
        hmm_error = get_mae_errors(hmm_preds, truth_values)
        lstm_error = get_mae_errors(lstm_preds, truth_values) 

        weights = normalized_inverse_of_errors_weighting(arima_error, lstm_error, hmm_error) # Weighting algorithm

        predictions.append(weights[0]*arima_pred + weights[1]*lstm_pred + weights[2]*hmm_pred)
        history = np.append(history,test[i])
        
        if i != len(test)-1:
            hmm_history = np.append(hmm_history,test_hmm[i]).reshape(-1,1)
    
    return predictions, arima_preds, hmm_preds, lstm_preds
        

In [49]:
de_preds, arima_preds, hmm_preds, lstm_preds = dynamic_ensemble_prediction(np.array(df_train.y), np.array(df_test.y))

1/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
4/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
5/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
6/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
7/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
8/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
9/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
10/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
11/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
12/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
13/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
14/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
15/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
16/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
18/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
19/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
20/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
21/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
22/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
23/412
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/s

In [30]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math
print('Dynamic Ensemble')
print(f'R2 Score : {r2_score(df_test.y,de_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,de_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,de_preds)}')
print('ARIMA')
print(f'R2 Score : {r2_score(df_test.y,arima_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,arima_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,arima_preds)}')
print('HMM')
print(f'R2 Score : {r2_score(df_test.y,hmm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,hmm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,hmm_preds)}')
print('LSTM')
print(f'R2 Score : {r2_score(df_test.y,lstm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y, lstm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,lstm_preds)}')

Dynamic Ensemble
R2 Score : 0.881727148731872
RMSE : 0.1076255338501715
MAE : 0.03887806263323038
ARIMA
R2 Score : 0.6218432803424067
RMSE : 0.19244591475690198
MAE : 0.061720508731204084
HMM
R2 Score : 0.5078220076674251
RMSE : 0.21955022196728474
MAE : 0.07231228600280035
LSTM
R2 Score : 0.9104949883515521
RMSE : 0.09362599842434782
MAE : 0.025956490960659336


In [31]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], mode='lines', name='Real Data'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_preds, mode='lines', name='Proposed Method'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_preds, mode='lines', name='ARIMA'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_preds, mode='lines', name='HMM'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_preds, mode='lines', name='LSTM'))
fig.add_trace(go.Scatter(x=df_train['ds'], y=df_train['y'], mode='lines', name='Training'))
fig.show()

In [32]:
# Getting Errors for each prediction of each method
test_df_list = list(df_test['y'])
arima_error = []
hmm_error = []
lstm_error = []
de_error = []
for i in range(len(test_df_list)):
    arima_error.append(mean_absolute_error([test_df_list[i]],[arima_preds[i]]))
    hmm_error.append(mean_absolute_error([test_df_list[i]],[hmm_preds[i]]))
    lstm_error.append(mean_absolute_error([test_df_list[i]],[lstm_preds[i]]))
    de_error.append(mean_absolute_error([test_df_list[i]],[de_preds[i]]))

In [33]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG Error comparison")
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_error, mode='lines', name='ARIMA Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_error, mode='lines', name='HMM Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_error, mode='lines', name='LSTM Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

## Other Ensembles

### Ensemble Mean

In [34]:
ensemble_mean_preds = []
ensemble_mean_errors = []
for i in range(len(arima_preds)):
    ensemble_mean_preds.append((arima_preds[i]+hmm_preds[i]+lstm_preds[i])/3)
    ensemble_mean_errors.append(mean_absolute_error([ensemble_mean_preds[i]],[test_df_list[i]]))

print('Ensemble Mean')
print(f'R2 Score : {r2_score(df_test.y,ensemble_mean_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,ensemble_mean_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,ensemble_mean_preds)}')

Ensemble Mean
R2 Score : 0.7659356774657744
RMSE : 0.15140506215388133
MAE : 0.04626695733494867


In [35]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Mean Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = ensemble_mean_errors, mode='lines', name='Mean Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Best model in hindsight

In [36]:
from sklearn.metrics import mean_absolute_percentage_error

bmh_preds = []
bmh_errors = []
mape = [mean_absolute_percentage_error(arima_preds, df_test['y']), mean_absolute_percentage_error(hmm_preds, df_test['y']), mean_absolute_percentage_error(lstm_preds, df_test['y'])]
bmh_preds = [arima_preds,hmm_preds,lstm_preds][np.argmin(mape)]

for i in range(len(test_df_list)):
    bmh_errors.append(mean_absolute_error([bmh_preds[i]],[test_df_list[i]]))

print('Best Model in Hindsight')
print(f'R2 Score : {r2_score(df_test.y,bmh_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,bmh_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,bmh_preds)}')

Best Model in Hindsight
R2 Score : 0.9104949883515521
RMSE : 0.09362599842434782
MAE : 0.025956490960659336


In [37]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with BMH Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = bmh_errors, mode='lines', name='BMH Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Ridge Ensemble

In [38]:
import numpy as np
from sklearn.linear_model import Ridge

def ridge_ensemble(forecasts, actuals, alpha=1.0):
    X = np.array(forecasts).T  
    y = np.array(actuals)       

    ridge = Ridge(alpha=alpha, fit_intercept=False)
    ridge.fit(X, y)
    
    weights = ridge.coef_
    ensemble_prediction = np.dot(X, weights)

    return {"weights": weights, "ensemble_prediction": ensemble_prediction}

In [39]:
forecasts = [arima_preds, hmm_preds, lstm_preds]
ridge_output = ridge_ensemble(forecasts, test_df_list)
ridge_preds = ridge_output['ensemble_prediction']
ridge_errors = []
for i in range(len(test_df_list)):
    ridge_errors.append(mean_absolute_error([ridge_preds[i]],[test_df_list[i]]))

print('Ridge Ensemble')
print(f'R2 Score : {r2_score(df_test.y,ridge_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,ridge_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,ridge_preds)}')

Ridge Ensemble
R2 Score : 0.9095630208352103
RMSE : 0.09411217469567396
MAE : 0.026426173130263388


### Exp3 Ensemble

In [40]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Ridge Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = ridge_errors, mode='lines', name='Ridge Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

In [41]:
class Exp3Ensemble:
    def __init__(self, num_models, window_size=1):
        self.num_models = num_models
        self.window_size = window_size
        self.regret = np.zeros(num_models)  # Initial regret for all models
        self.weights = np.ones(num_models) / num_models  # Initial equal weights

    def update(self, forecasts, actuals, t):
        if t < self.window_size:
            return 

        for i in range(self.num_models):
            self.regret[i] = np.sum([(actuals[s] - forecasts[i][s]) ** 2 for s in range(t - self.window_size, t)])

        eta_t = np.sqrt(8 * np.log(self.num_models) / self.window_size)
        new_weights = np.exp(-eta_t * self.regret)
        if np.sum(new_weights) == 0:
            new_weights = np.ones(self.num_models)/self.num_models
        self.weights = new_weights / np.sum(new_weights)

    def get_ensemble_prediction(self, forecasts):
        forecasts = np.array(forecasts).flatten()
        return np.dot(self.weights, forecasts)

num_models = 3
window_size = 2
exp3_ensemble = Exp3Ensemble(num_models, window_size)

forecasts = [arima_preds, hmm_preds, lstm_preds]

exp3_preds = []
for t in range(len(test_df_list)):
    exp3_ensemble.update(forecasts, test_df_list, t)
    exp3_preds.append(exp3_ensemble.get_ensemble_prediction([[model[t]] for model in forecasts]))

exp3_errors = []
for i in range(len(test_df_list)):
    exp3_errors.append(mean_absolute_error([exp3_preds[i]],[test_df_list[i]]))

In [42]:
print('Exp3 Ensemble')
print(f'R2 Score : {r2_score(df_test.y,exp3_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,exp3_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,exp3_preds)}')

Exp3 Ensemble
R2 Score : 0.7937623497180643
RMSE : 0.14212051132227196
MAE : 0.04426214183212914


In [43]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Exp3 Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = exp3_errors, mode='lines', name='Exp3 Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Passive Aggressive Ensemble

In [44]:
import numpy as np

class PassiveAggressiveForecaster:
    def __init__(self, num_models, epsilon=2):
        self.epsilon = epsilon  # Margin parameter
        self.beta = np.ones(num_models) / num_models  # Initialize weights equally

    def update_weights(self, X_t, y_t):
        X_t = np.array(X_t)  # Ensure it's a NumPy array
        y_t_pred = np.dot(X_t, self.beta)  # Weighted prediction

        loss = abs(y_t_pred - y_t) - self.epsilon
        tau_t = max(0, loss) / (np.linalg.norm(X_t) ** 2 + 1e-8)  # Small term to avoid division by zero

        self.beta += np.sign(y_t_pred - y_t) * tau_t * X_t

        self.beta = np.maximum(self.beta, 0)  # Ensure non-negative weights
        self.beta /= np.sum(self.beta)  # Normalize weights

    def predict(self, X_t):
        return np.dot(X_t, self.beta)


num_models = 3

# Hyper parameter tuning
maxr2 = 0
maxep=-1
for eps in np.arange(0.1,50,0.2):
    forecaster = PassiveAggressiveForecaster(num_models,eps)
    pa_preds = []
    for t in range(len(test_df_list)):
        X_t = [model[t] for model in forecasts] 
        y_t = test_df_list[t]  
        pa_preds.append(forecaster.predict(X_t))
        forecaster.update_weights(X_t, y_t)
    
    if maxr2 < r2_score(df_test.y,pa_preds):
        maxep = eps
        maxr2 = r2_score(df_test.y,pa_preds)
    
forecaster = PassiveAggressiveForecaster(num_models,maxep)
pa_preds = []
for t in range(len(test_df_list)):
    X_t = [model[t] for model in forecasts]  # Model predictions at time t
    y_t = test_df_list[t]  # Actual value at time t

    # Predict and update weights
    pa_preds.append(forecaster.predict(X_t))
    forecaster.update_weights(X_t, y_t)

pa_errors = []
for i in range(len(test_df_list)):
    pa_errors.append(mean_absolute_error([pa_preds[i]],[test_df_list[i]]))

In [45]:
print('Passive Aggressive Ensemble')
print(f'R2 Score : {r2_score(df_test.y,pa_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,pa_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,pa_preds)}')

Passive Aggressive Ensemble
R2 Score : 0.7659356774657744
RMSE : 0.15140506215388133
MAE : 0.04626695733494867
